In [61]:
import pandas as pd
import json
import plotly.express as px


In [11]:
data = json.load(
    open("../../../Downloads/maarten-thesis-all_annotations.json"))
examples = data["examples"]


In [30]:
Examples = pd.DataFrame(examples,columns=['content','metadata','example_id'])
all_classifications = sum(map(lambda x:x['classifications'],examples),[])
class_data = pd.io.json.json_normalize(all_classifications, meta=[
                                       'example_id', 'classname'], record_path='classified_by')

Classifications = pd.io.json.json_normalize(all_classifications,meta=['example_id','classname'],record_path='classified_by')
ClassPivot = Classifications.pivot_table(index='example_id',columns='classname',values='annotator_id',aggfunc=len).fillna(0)
ExamplesWithClasses = pd.merge(Examples,ClassPivot,on='example_id')
ExamplesWithClasses = ExamplesWithClasses.set_index(['example_id','content']).sort_index()


C:\Users\maart\AppData\Local\Temp/ipykernel_7160/80591669.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  class_data = pd.io.json.json_normalize(all_classifications, meta=[
C:\Users\maart\AppData\Local\Temp/ipykernel_7160/80591669.py:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  Classifications = pd.io.json.json_normalize(all_classifications,meta=['example_id','classname'],record_path='classified_by')


In [46]:
ExamplesWithClasses.head()
data_classes = ExamplesWithClasses.reset_index()[["content", "Funny", "Incoherent", "Not Funny"]].set_index("content").stack().reset_index()
data_classes.columns = ["title", "class", "value"]
data_classes = data_classes[data_classes.value==1].drop("value", axis=1)
data_classes.head()

,title,class
0,Trump Claims He Won German Election,Funny
4,Amy Coney Barrett Appears Before Brett Kavanau...,Incoherent
8,Trump Announces All NPR Policies Will Apply To...,Not Funny
9,Hannity Sent Texts Warning Trump That Capitol ...,Funny
14,Trump Sues Eighty-one Million Voters for Banni...,Not Funny


In [58]:
# Read generated dataset
generated = pd.read_excel("../../../Downloads/outputs (2).xlsx")
generated = generated.drop("Unnamed: 0", axis=1).stack().reset_index().drop("level_0", axis=1)
generated.columns = ["type", "title"]
generated

,type,title
0,keywords,Jackson | Kavanaugh
1,original,Jackson Refuses Kavanaugh’s Invitation to Beer...
2,pred_large,The Truth Is Uncovering The True Kavanaugh Ass...
3,pred_l_pos,Jackson Immediately Responds To Kavanaugh Dema...
4,pred_base,Jackson Turns 35 And Kavanaugh Refuses To Start.
...,...,...
1195,original,P.A. System at Trump Rally Accidentally Blasts...
1196,pred_large,Trump Unveils Sprawling Woodward Mantel.
1197,pred_l_pos,Trump Condemns Woodward As Not Satisfying Exis...
1198,pred_base,Trump Meets With Woodward.


In [63]:
merged = pd.merge(left=data_classes, right=generated, how="left", right_on="title", left_on="title")
merged.head()

,title,class,type
0,Trump Claims He Won German Election,Funny,original
1,Amy Coney Barrett Appears Before Brett Kavanau...,Incoherent,pred_base
2,Trump Announces All NPR Policies Will Apply To...,Not Funny,pred_large
3,Hannity Sent Texts Warning Trump That Capitol ...,Funny,original
4,Trump Sues Eighty-one Million Voters for Banni...,Not Funny,original


In [64]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 211 entries, 0 to 210
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   211 non-null    object
 1   class   211 non-null    object
 2   type    211 non-null    object
dtypes: object(3)
memory usage: 6.6+ KB


In [72]:
px.bar(merged.groupby(["type", "class"]).count().reset_index(), x="type", y="title", color="class", barmode="group")